<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
from __future__ import print_function

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
import pandas as pd
import numpy as np
import requests
import random
import sys
import os

In [2]:
text = requests.get('https://www.gutenberg.org/files/100/100-0.txt')

In [3]:
text = text.text # lol
text = text.replace('\r', '')

In [4]:
len(text)

5610465

In [5]:
chars = list(set(text))

# First 5
chars[:5]

['7', '%', '"', 'w', '!']

In [6]:
char_int = {c:i for i, c in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

In [7]:
len(chars)

113

In [8]:
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 80 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  1122085


In [9]:
# Creates a matrix of Falses of shape 223071, 40, 65
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)

# Creates a matrix of Falses of shape 223071, 1, 65
y = np.zeros((len(sequences),  len(chars)), dtype=np.bool)


# Loop through the entire text
for i, sequence in enumerate(sequences):

    # loop through a sequence
    for t, char in enumerate(sequence):

        # encode the char as true (rest stay false)
        x[i,t,char] = 1
        
    # encode the target chars     
    y[i, next_char[i]] = 1

In [10]:
model = Sequential()

model.add(LSTM(128, input_shape=(maxlen, len(chars)), activation='tanh',
          recurrent_activation='sigmoid', dropout=0.2, recurrent_dropout=0,
          unroll=False, use_bias=True))

model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [12]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [13]:
# fit the model
         
model.fit(x, y,
          batch_size=512,
          epochs=25,
          callbacks=[print_callback])

Epoch 1/25
2192/2192 [==============================] - ETA: 0s - loss: 2.6418
----- Generating text after Epoch: 0
----- Generating with seed: "LES. Was Mahomet inspired with a dove?
 "
LES. Was Mahomet inspired with a dove?
          wives by hat, maclinkexil hh toe ow dathorpinSjin.
Fo  fice tingusose hacke tour
     mo in seryep  f fere hress Hariee'd! to manicp omy
Tie sore his misct.    TI thay my in yeas nd ly qanawnurs; bu bigr, tiuse dotl.
Tou Twelly,
  Wlllw a taos ul mare,
Neh loth storotse  ald terirebusin

    hue th le sioht. Chelp nnthe nent
f o nard,spondervitein of sinmfitie,
Whlt asorn f this the arofr
2192/2192 [==============================] - 906s 413ms/step - loss: 2.6418
Epoch 2/25
2192/2192 [==============================] - ETA: 0s - loss: 2.2900
----- Generating text after Epoch: 1
----- Generating with seed: "  FIRST SERVANT. They have made him drin"
  FIRST SERVANT. They have made him drinithe wwock theat,
    arvieisteer shouk ono I
  hants oh.
Bo tiat; bu

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN